# Med-GrapherNCA: Single-Level Experiments
### Pixel-Grapher-NCA (m1), Patch-Grapher-NCA (m2), Baseline BackboneNCA (b1)

This notebook trains and evaluates individual models at a single resolution level on the ISIC 2018 dataset.

---

## 0. Google Colab Setup: Mount Drive & Download Dataset

In [ ]:
print("hi")


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Setup project directory in Google Drive
PROJECT_ROOT = '/content/drive/MyDrive/Experiments/Grapher_NCA'
DATASET_DIR = os.path.join(PROJECT_ROOT, 'datasets', 'ISIC2018')
MODELS_DIR = os.path.join(PROJECT_ROOT, 'Models')

os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

print(f'Project root: {PROJECT_ROOT}')
print(f'Dataset dir:  {DATASET_DIR}')
print(f'Models dir:   {MODELS_DIR}')

In [ ]:
%%bash -s "$DATASET_DIR"
DATASET_DIR=$1

# Download ISIC 2018 Task 1 Training Data (images + masks)
if [ ! -d "$DATASET_DIR/ISIC2018_Task1-2_Training_Input" ]; then
    echo "Downloading ISIC 2018 training images..."
    wget -q --show-progress -O "$DATASET_DIR/train_input.zip" \
        "https://isic-archive.s3.amazonaws.com/challenges/2018/ISIC2018_Task1-2_Training_Input.zip"
    echo "Extracting training images..."
    unzip -q "$DATASET_DIR/train_input.zip" -d "$DATASET_DIR"
    rm "$DATASET_DIR/train_input.zip"
else
    echo "Training images already exist, skipping download."
fi

if [ ! -d "$DATASET_DIR/ISIC2018_Task1_Training_GroundTruth" ]; then
    echo "Downloading ISIC 2018 training ground truth..."
    wget -q --show-progress -O "$DATASET_DIR/train_gt.zip" \
        "https://isic-archive.s3.amazonaws.com/challenges/2018/ISIC2018_Task1_Training_GroundTruth.zip"
    echo "Extracting training ground truth..."
    unzip -q "$DATASET_DIR/train_gt.zip" -d "$DATASET_DIR"
    rm "$DATASET_DIR/train_gt.zip"
else
    echo "Training ground truth already exists, skipping download."
fi

# Download ISIC 2018 Task 1 Test Data (images + masks)
if [ ! -d "$DATASET_DIR/ISIC2018_Task1-2_Test_Input" ]; then
    echo "Downloading ISIC 2018 test images..."
    wget -q --show-progress -O "$DATASET_DIR/test_input.zip" \
        "https://isic-archive.s3.amazonaws.com/challenges/2018/ISIC2018_Task1-2_Test_Input.zip"
    echo "Extracting test images..."
    unzip -q "$DATASET_DIR/test_input.zip" -d "$DATASET_DIR"
    rm "$DATASET_DIR/test_input.zip"
else
    echo "Test images already exist, skipping download."
fi

if [ ! -d "$DATASET_DIR/ISIC2018_Task1_Test_GroundTruth" ]; then
    echo "Downloading ISIC 2018 test ground truth..."
    wget -q --show-progress -O "$DATASET_DIR/test_gt.zip" \
        "https://isic-archive.s3.amazonaws.com/challenges/2018/ISIC2018_Task1_Test_GroundTruth.zip"
    echo "Extracting test ground truth..."
    unzip -q "$DATASET_DIR/test_gt.zip" -d "$DATASET_DIR"
    rm "$DATASET_DIR/test_gt.zip"
else
    echo "Test ground truth already exists, skipping download."
fi

echo "\nDataset contents:"
ls -la "$DATASET_DIR"

In [ ]:
import os, sys

# Clone repo from GitHub into Colab's /content (fast, always fresh)
REPO_DIR = '/content/grapher-nca/M3D-NCA'
if not os.path.isdir('/content/grapher-nca'):
    os.system('git clone https://github.com/AvniMittal13/grapher-nca.git /content/grapher-nca')

if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

# Install dependencies
os.system('pip install -q torchio==0.18.82 nibabel tensorboard')

print(f'Repo dir: {REPO_DIR}')
print(f'sys.path OK: {REPO_DIR in sys.path}')

## 1. Imports

In [ ]:
import torch
from src.datasets.ISIC_Dataset import ISIC2018_Dataset
from src.models.Model_BackboneNCA import BackboneNCA
from src.models.Model_GrapherNCA_M1 import GrapherNCA_M1
from src.models.Model_GrapherNCA_M2 import GrapherNCA_M2
from src.losses.LossFunctions import DiceLoss
from src.utils.Experiment import Experiment
from src.agents.Agent_Med_NCA import Agent_Med_NCA

## 2. Configure Experiment

Set `model_type` to choose which model to train: `'b1'`, `'m1'`, or `'m2'`

In [ ]:
# Choose model type: 'b1' (BackboneNCA), 'm1' (Pixel-Grapher), 'm2' (Patch-Grapher)
model_type = 'm1'

config = [{
    'img_path': os.path.join(DATASET_DIR, 'ISIC2018_Task1-2_Training_Input'),
    'label_path': os.path.join(DATASET_DIR, 'ISIC2018_Task1_Training_GroundTruth'),
    'model_path': os.path.join(MODELS_DIR, f'GrapherNCA_single_{model_type}'),
    'device': 'cuda:0',
    'unlock_CPU': True,
    # Optimizer
    'lr': 1e-4,
    'lr_gamma': 0.9999,
    'betas': (0.5, 0.5),
    # Training
    'save_interval': 10,
    'evaluate_interval': 10,
    'n_epoch': 1000,
    'batch_size': 8,
    # Model
    'channel_n': 64,
    'inference_steps': 10,
    'cell_fire_rate': 0.5,
    'input_channels': 3,  # RGB for ISIC
    'output_channels': 1,
    'hidden_size': 512,
    'train_model': 0,  # Single level
    # Data
    'input_size': [(256, 256)],
    'data_split': [0.7, 0, 0.3],
}]

## 3. Choose Architecture and Setup

In [ ]:
dataset = ISIC2018_Dataset(input_channels=config[0]['input_channels'])
device = torch.device(config[0]['device'])

# Create model based on model_type
if model_type == 'b1':
    ca = BackboneNCA(
        config[0]['channel_n'], config[0]['cell_fire_rate'], device,
        hidden_size=config[0]['hidden_size'], input_channels=config[0]['input_channels']
    ).to(device)
elif model_type == 'm1':
    ca = GrapherNCA_M1(
        config[0]['channel_n'], config[0]['cell_fire_rate'], device,
        hidden_size=config[0]['hidden_size'], input_channels=config[0]['input_channels'],
        k=9
    ).to(device)
elif model_type == 'm2':
    ca = GrapherNCA_M2(
        config[0]['channel_n'], config[0]['cell_fire_rate'], device,
        hidden_size=config[0]['hidden_size'], input_channels=config[0]['input_channels'],
        k=9, patch_size=4
    ).to(device)

ca = [ca]  # Single level, wrapped in list for Agent_Med_NCA
agent = Agent_Med_NCA(ca)
exp = Experiment(config, dataset, ca, agent)
dataset.set_experiment(exp)
exp.set_model_state('train')
data_loader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=exp.get_from_config('batch_size'))

loss_function = DiceLoss()

# Print model parameter count
total_params = sum(p.numel() for p in ca[0].parameters())
print(f'Model type: {model_type}, Total parameters: {total_params}')

## 4. Train

In [ ]:
agent.train(data_loader, loss_function)

## 5. Evaluate

In [ ]:
agent.getAverageDiceScore(pseudo_ensemble=True, showResults=True)